<h1>Image Colorizer</h1><hr>
<h3>Jason Lombino - Capstone Project Submission</h3>

<h1>Business Proposal</h1>

Photoshop?<br>
Museum/Exhibit?<br>
Web App/Mobile App?

<h1>Background Information</h1>

<h3>Why Neural Networks?</h3>

<h3>Selected Architecture</h3>

<h3>Training Methodology and GANs</h3>


<h1>Data and Sources</h1>

https://www.kaggle.com/datasets/utkarshsaxenadn/landscape-recognition-image-dataset-12k-images

10k training, 1k validation (not all used/shown), 500 test (not all used/shown)

Validation and test split from test in source because not all were needed for testing.
Validation used in individual model notebooks test images shown here.

<h1>Training Losses</h1>

Tensorboard data goes here

<h1>Results</h1>

Models still need to be trained

In [1]:
import colorizerutils as utils
from tensorflow.keras.models import load_model

mse_gen = load_model('models/mse/generator.h5')
gan_gen = load_model('models/gan/generator.h5')

for test_batch in range(4):
    test_images = utils.image_loader(directory='../images/test_images/',
     batch_size=8,training=False).__getitem__(test_batch)
    utils.display_images(gray_channel=test_images[0],color_channels=test_images[1],
    generator1=mse_gen,gen1_title='MSE Colored',generator2=gan_gen,gen2_title='GAN Colored');

<h1>Conclusion and Future Work</h1>

Larger Images?
Different topics?

Success?